In [ ]:
# MYH7 Variant Analysis Pipeline

## Comprehensive analysis of MYH7 (myosin heavy chain 7) variants
### Combining ClinVar, AlphaMissense, AlphaFold, and Rosetta data

**Objectives:**
1. Collect MYH7 variants from ClinVar database
2. Obtain pathogenicity scores from AlphaMissense
3. Extract structural confidence (pLDDT) from AlphaFold
4. Calculate stability changes (ΔΔG) using Rosetta
5. Analyze correlations and generate insights


In [ ]:
# Install required packages
!pip install --quiet pandas numpy matplotlib seaborn requests biopython rdkit-pypi deepchem


In [ ]:
import pandas as pd
import numpy as np
import requests
import tempfile
import re
import os
from pathlib import Path
from Bio.PDB import PDBParser
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Create directories
Path("../data/variants").mkdir(parents=True, exist_ok=True)
Path("../results/figures").mkdir(parents=True, exist_ok=True)

# Amino acid mapping dictionaries
three_to_one = {
    "Ala": "A", "Cys": "C", "Asp": "D", "Glu": "E", "Phe": "F", "Gly": "G",
    "His": "H", "Ile": "I", "Lys": "K", "Leu": "L", "Met": "M", "Asn": "N",
    "Pro": "P", "Gln": "Q", "Arg": "R", "Ser": "S", "Thr": "T", "Val": "V",
    "Trp": "W", "Tyr": "Y"
}
one_to_three = {v: k for k, v in three_to_one.items()}

print("✓ Setup complete!")


In [ ]:
## 1. Load and Analyze Existing Dataset

First, let's load your existing comprehensive dataset and see what we're working with.


In [ ]:
# Load your existing dataset
data_file = "../MYH7_variants_final_annotated.csv"

if Path(data_file).exists():
    print(f"📁 Loading existing dataset: {data_file}")
    df = pd.read_csv(data_file)
    print(f"✓ Loaded {len(df)} variants")
    
    # Display basic info
    print(f"\n📊 Dataset Overview:")
    print(f"- Total variants: {len(df):,}")
    print(f"- Columns: {list(df.columns)}")
    print(f"- Unique protein changes: {df['ProteinChange'].nunique() if 'ProteinChange' in df.columns else 'N/A'}")
    
    # Show first few rows
    print(f"\n🔍 First 5 rows:")
    display(df.head())
    
    # Basic statistics
    print(f"\n📈 Score Statistics:")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if col in ['AlphaMissense', 'Rosetta_ddG', 'pLDDT']:
            stats = df[col].describe()
            print(f"- {col}: mean={stats['mean']:.3f}, std={stats['std']:.3f}, range=[{stats['min']:.3f}, {stats['max']:.3f}]")
    
    # Check for missing data
    print(f"\n🔍 Missing Data Summary:")
    missing_summary = df.isnull().sum().sort_values(ascending=False)
    for col, missing_count in missing_summary.items():
        if missing_count > 0:
            print(f"- {col}: {missing_count} missing ({missing_count/len(df)*100:.1f}%)")
else:
    print(f"❌ Dataset not found at {data_file}")
    print("Let's fetch the data from scratch...")
    df = None


In [ ]:
## 2. Comprehensive Data Analysis & Visualization

Let's create a comprehensive visualization suite to understand the relationships between pathogenicity, structural confidence, and stability.


In [ ]:
if df is not None:
    # Create comprehensive visualization suite
    fig = plt.figure(figsize=(20, 15))
    
    # 1. AlphaMissense vs Rosetta ΔΔG (your main correlation)
    ax1 = plt.subplot(3, 3, 1)
    if 'AlphaMissense' in df.columns and 'Rosetta_ddG' in df.columns:
        mask = df['AlphaMissense'].notna() & df['Rosetta_ddG'].notna()
        plt.scatter(df.loc[mask, 'AlphaMissense'], df.loc[mask, 'Rosetta_ddG'], 
                   alpha=0.6, c='steelblue', s=40)
        plt.xlabel('AlphaMissense Pathogenicity Score')
        plt.ylabel('Rosetta ΔΔG (REU)')
        plt.title('🎯 Pathogenicity vs Structural Stability')
        plt.grid(True, alpha=0.3)
        
        # Calculate and display correlation
        corr = df['AlphaMissense'].corr(df['Rosetta_ddG'])
        plt.text(0.05, 0.95, f'Pearson r = {corr:.3f}', transform=ax1.transAxes,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        print(f"🔬 Key Finding: AlphaMissense vs Rosetta ΔΔG correlation: r = {corr:.3f}")
    
    # 2. pLDDT confidence scores distribution
    ax2 = plt.subplot(3, 3, 2)
    if 'pLDDT' in df.columns:
        df['pLDDT'].hist(bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
        plt.axvline(df['pLDDT'].mean(), color='red', linestyle='--', linewidth=2,
                   label=f'Mean: {df["pLDDT"].mean():.1f}')
        plt.xlabel('AlphaFold Confidence (pLDDT)')
        plt.ylabel('Number of Variants')
        plt.title('📊 Structural Confidence Distribution')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    # 3. AlphaMissense distribution
    ax3 = plt.subplot(3, 3, 3)
    if 'AlphaMissense' in df.columns:
        df['AlphaMissense'].hist(bins=50, alpha=0.7, color='coral', edgecolor='black')
        plt.axvline(df['AlphaMissense'].mean(), color='darkred', linestyle='--', linewidth=2,
                   label=f'Mean: {df["AlphaMissense"].mean():.3f}')
        plt.axvline(0.8, color='red', linestyle=':', linewidth=2, label='High Pathogenicity (0.8)')
        plt.xlabel('AlphaMissense Pathogenicity Score')
        plt.ylabel('Number of Variants')
        plt.title('🧬 Pathogenicity Score Distribution')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    # 4. Rosetta ΔΔG distribution with stability interpretation
    ax4 = plt.subplot(3, 3, 4)
    if 'Rosetta_ddG' in df.columns:
        mask = df['Rosetta_ddG'].notna()
        df.loc[mask, 'Rosetta_ddG'].hist(bins=40, alpha=0.7, color='orange', edgecolor='black')
        plt.axvline(0, color='red', linestyle='-', linewidth=3, label='Neutral')
        plt.axvline(1, color='darkred', linestyle='--', label='Destabilizing (+1 REU)')
        plt.axvline(-1, color='blue', linestyle='--', label='Stabilizing (-1 REU)')
        plt.xlabel('Rosetta ΔΔG (REU)')
        plt.ylabel('Number of Variants')
        plt.title('⚖️ Protein Stability Changes')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Add stability stats
        destabilizing = (df['Rosetta_ddG'] > 1).sum()
        stabilizing = (df['Rosetta_ddG'] < -1).sum()
        neutral = ((df['Rosetta_ddG'] >= -1) & (df['Rosetta_ddG'] <= 1)).sum()
        print(f"⚖️  Stability Analysis: {destabilizing} destabilizing, {neutral} neutral, {stabilizing} stabilizing")
    
    # 5. pLDDT vs AlphaMissense with residue position coloring
    ax5 = plt.subplot(3, 3, 5)
    if 'pLDDT' in df.columns and 'AlphaMissense' in df.columns and 'Residue' in df.columns:
        scatter = plt.scatter(df['pLDDT'], df['AlphaMissense'], 
                            alpha=0.6, c=df['Residue'], cmap='viridis', s=30)
        plt.xlabel('AlphaFold Confidence (pLDDT)')
        plt.ylabel('AlphaMissense Score')
        plt.title('🗺️ Confidence vs Pathogenicity (by Position)')
        plt.colorbar(scatter, label='Residue Position')
        plt.grid(True, alpha=0.3)
        
        # Add quadrant lines
        plt.axhline(0.8, color='red', linestyle=':', alpha=0.5)
        plt.axvline(90, color='red', linestyle=':', alpha=0.5)
    
    # 6. High-impact variants (high pathogenicity + destabilizing)
    ax6 = plt.subplot(3, 3, 6)
    if all(col in df.columns for col in ['AlphaMissense', 'Rosetta_ddG']):
        high_path = df['AlphaMissense'] > 0.8
        destabilizing = df['Rosetta_ddG'] > 1.0
        
        colors = ['lightgray' if not (hp and dest) else 'red' 
                 for hp, dest in zip(high_path, destabilizing)]
        
        plt.scatter(df['AlphaMissense'], df['Rosetta_ddG'], c=colors, alpha=0.7, s=30)
        plt.xlabel('AlphaMissense Score')
        plt.ylabel('Rosetta ΔΔG (REU)')
        plt.title('🚨 High-Impact Variants')
        plt.axhline(1.0, color='red', linestyle='--', alpha=0.5, label='Destabilizing')
        plt.axvline(0.8, color='red', linestyle='--', alpha=0.5, label='High Pathogenicity')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Count high-impact variants
        high_impact = (high_path & destabilizing).sum()
        print(f"🚨 High-Impact Variants: {high_impact} variants with AlphaMissense>0.8 AND ΔΔG>1.0")
    
    # 7. Residue position vs pathogenicity (sequence analysis)
    ax7 = plt.subplot(3, 3, 7)
    if 'Residue' in df.columns and 'AlphaMissense' in df.columns:
        plt.scatter(df['Residue'], df['AlphaMissense'], alpha=0.5, s=20, c='purple')
        plt.xlabel('Residue Position')
        plt.ylabel('AlphaMissense Score')
        plt.title('🧪 Pathogenicity Along Sequence')
        plt.axhline(0.8, color='red', linestyle='--', alpha=0.5)
        plt.grid(True, alpha=0.3)
    
    # 8. Correlation heatmap
    ax8 = plt.subplot(3, 3, 8)
    numeric_cols = ['AlphaMissense', 'Rosetta_ddG', 'pLDDT', 'Residue']
    available_cols = [col for col in numeric_cols if col in df.columns]
    
    if len(available_cols) >= 2:
        corr_matrix = df[available_cols].corr()
        sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0, 
                   square=True, cbar_kws={'label': 'Correlation'})
        plt.title('🔗 Correlation Matrix')
        
    # 9. Top variants table
    ax9 = plt.subplot(3, 3, 9)
    ax9.axis('tight')
    ax9.axis('off')
    
    if 'AlphaMissense' in df.columns:
        top_variants = df.nlargest(10, 'AlphaMissense')
        table_data = []
        for _, row in top_variants.iterrows():
            variant = row.get('ProteinChange', 'N/A')
            am_score = f"{row['AlphaMissense']:.3f}" if pd.notna(row['AlphaMissense']) else 'N/A'
            ddg = f"{row['Rosetta_ddG']:.2f}" if 'Rosetta_ddG' in row and pd.notna(row['Rosetta_ddG']) else 'N/A'
            plddt = f"{row['pLDDT']:.1f}" if 'pLDDT' in row and pd.notna(row['pLDDT']) else 'N/A'
            table_data.append([variant, am_score, ddg, plddt])
        
        table = ax9.table(cellText=table_data[:8],  # Show top 8 to fit
                          colLabels=['Variant', 'AlphaMissense', 'ΔΔG', 'pLDDT'],
                          cellLoc='center', loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(8)
        table.scale(1, 1.5)
        ax9.set_title('🏆 Top Pathogenic Variants', pad=20)
    
    plt.tight_layout()
    plt.savefig('../results/figures/comprehensive_myh7_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Comprehensive analysis complete!")
    print(f"📊 Visualization saved to ../results/figures/comprehensive_myh7_analysis.png")
else:
    print("❌ No data loaded - run data collection first")


In [ ]:
## 3. Fix Rosetta ΔΔG Calculations 

Let's fix the Rosetta command line issues and run proper ΔΔG calculations for your top variants.


In [ ]:
def create_fixed_rosetta_analysis(top_n=20):
    """
    Create a robust Rosetta ΔΔG analysis with fixed command line options
    """
    
    # First, let's identify top variants for analysis
    if df is not None and 'AlphaMissense' in df.columns:
        print(f"🎯 Selecting top {top_n} variants by AlphaMissense score...")
        top_variants = df.nlargest(top_n, 'AlphaMissense')
        
        print("📋 Top variants selected:")
        for i, (_, row) in enumerate(top_variants.iterrows(), 1):
            variant = row.get('ProteinChange', 'N/A')
            score = row.get('AlphaMissense', 0)
            print(f"{i:2d}. {variant}: {score:.3f}")
        
        # Create mutation file with proper format
        mutfile_content = []
        print(f"\n🔧 Creating Rosetta mutation file...")
        
        for _, row in top_variants.iterrows():
            protein_change = row.get('ProteinChange', '')
            if len(protein_change) >= 6:  # e.g., "Arg403Gln"
                # Extract components
                wt_aa = protein_change[:3]  # e.g., "Arg"
                position = protein_change[3:-3]  # e.g., "403" 
                mut_aa = protein_change[-3:]  # e.g., "Gln"
                
                # Convert to single letter codes
                wt_single = three_to_one.get(wt_aa, "X")
                mut_single = three_to_one.get(mut_aa, "X")
                
                if wt_single != "X" and mut_single != "X":
                    mutfile_content.append(f"A {position} {wt_single} {mut_single}")
                    print(f"  ✓ {protein_change} -> A {position} {wt_single} {mut_single}")
        
        # Save mutation file
        mutfile_path = "../top_variants_fixed.txt"
        with open(mutfile_path, 'w') as f:
            f.write("\\n".join(mutfile_content))
        
        print(f"✅ Mutation file saved: {mutfile_path}")
        print(f"📊 {len(mutfile_content)} variants ready for Rosetta analysis")
        
        return mutfile_path, mutfile_content
    else:
        print("❌ No data available for Rosetta analysis")
        return None, None

# Create the fixed analysis
mutfile_path, mutations = create_fixed_rosetta_analysis()


In [ ]:
# Create the fixed Rosetta command script
if mutations is not None:
    print("🔧 Creating fixed Rosetta execution script...")
    
    bash_script = '''#!/bin/bash
# Fixed Rosetta ΔΔG Analysis Script
# Addresses the command line option errors from ROSETTA_CRASH.log

echo "🚀 Starting Rosetta ΔΔG analysis..."

# Ensure Rosetta executable is available and executable
if [ ! -f "ddg_monomer.linuxgccrelease" ]; then
    echo "❌ Error: ddg_monomer.linuxgccrelease not found"
    echo "Please ensure Rosetta is properly installed"
    exit 1
fi

chmod +x ddg_monomer.linuxgccrelease

# Check if PDB file exists
if [ ! -f "MYH7_native.pdb" ]; then
    echo "❌ Error: MYH7_native.pdb not found"
    echo "Please ensure the structure file is available"
    exit 1
fi

# Process each mutation with FIXED command line options
mutation_count=0
success_count=0

while read -r CHAIN POS WT MUT; do
    if [ -z "$CHAIN" ] || [ -z "$POS" ] || [ -z "$WT" ] || [ -z "$MUT" ]; then
        continue
    fi
    
    mutation_count=$((mutation_count + 1))
    VARIANT_NAME="${CHAIN}${POS}${WT}${MUT}"
    echo "🧬 Processing variant $mutation_count: $VARIANT_NAME"
    
    # Create individual mutation file
    echo "$CHAIN $POS $WT $MUT" > temp_mut_${VARIANT_NAME}.txt
    
    # FIXED Rosetta command (addressing the crash log issues)
    ./ddg_monomer.linuxgccrelease \\
        -s MYH7_native.pdb \\
        -ddg:mutfile temp_mut_${VARIANT_NAME}.txt \\
        -ddg:iterations 3 \\
        -out:file:scorefile score_${VARIANT_NAME}.sc \\
        -ddg:dump_pdbs false \\
        -ignore_unrecognized_res \\
        -mute all \\
        > rosetta_log_${VARIANT_NAME}.txt 2>&1
    
    if [ $? -eq 0 ]; then
        echo "  ✅ Success: $VARIANT_NAME"
        success_count=$((success_count + 1))
    else
        echo "  ❌ Failed: $VARIANT_NAME"
        echo "  Check rosetta_log_${VARIANT_NAME}.txt for details"
    fi
    
    # Cleanup temp mutation file
    rm -f temp_mut_${VARIANT_NAME}.txt
    
done < top_variants_fixed.txt

echo ""
echo "📊 Analysis complete!"
echo "   Total mutations: $mutation_count"
echo "   Successful: $success_count"
echo "   Failed: $((mutation_count - success_count))"

# Combine all score files
if [ $success_count -gt 0 ]; then
    echo "📋 Combining score files..."
    
    # Create header from first score file
    first_score=$(ls score_*.sc 2>/dev/null | head -1)
    if [ ! -z "$first_score" ]; then
        head -1 "$first_score" > combined_scores.sc
        
        # Add data from all score files (skip headers)
        for score_file in score_*.sc; do
            if [ -f "$score_file" ]; then
                tail -n +2 "$score_file" >> combined_scores.sc
            fi
        done
        
        echo "✅ Combined scores saved to: combined_scores.sc"
    fi
else
    echo "❌ No successful calculations to combine"
fi

echo "🎯 Rosetta ΔΔG analysis finished!"
'''
    
    # Save the script
    script_path = "../run_rosetta_fixed.sh"
    with open(script_path, 'w') as f:
        f.write(bash_script)
    
    print(f"✅ Fixed Rosetta script saved: {script_path}")
    print("\\n🔧 Key fixes applied:")
    print("  - Fixed option name: '-out:file:scorefile' (not '-ddg:scorefile')")
    print("  - Added proper error handling")
    print("  - Individual mutation processing")
    print("  - Automatic score file combination")
    print("  - Detailed logging")
    
    print(f"\\n🚀 To run the analysis, execute:")
    print(f"   cd .. && bash run_rosetta_fixed.sh")
    
else:
    print("❌ Cannot create Rosetta script - no mutations available")


In [ ]:
## 4. Small Molecule Analysis Component

Let's add the molecular modeling component using QM9 dataset and DeepChem for potential therapeutic targeting.


In [ ]:
def setup_molecular_analysis():
    """
    Set up small molecule analysis pipeline for therapeutic targeting
    """
    try:
        import deepchem as dc
        from rdkit import Chem
        from rdkit.Chem import Descriptors, Crippen
        print("✅ DeepChem and RDKit successfully imported")
        
        # Load QM9 dataset for molecular property prediction
        print("🧪 Loading QM9 molecular dataset...")
        qm9_tasks, qm9_datasets, qm9_transformers = dc.molnet.load_qm9(
            featurizer='GraphConv', 
            reload=False,
            data_dir='../data/molecules'
        )
        
        train_dataset, valid_dataset, test_dataset = qm9_datasets
        
        print(f"📊 QM9 Dataset loaded:")
        print(f"  - Training: {len(train_dataset)} molecules")
        print(f"  - Validation: {len(valid_dataset)} molecules") 
        print(f"  - Test: {len(test_dataset)} molecules")
        print(f"  - Tasks: {qm9_tasks}")
        
        # Example molecular analysis - drug-like properties
        print("\\n💊 Analyzing drug-like properties...")
        
        # Sample some molecules for analysis
        sample_molecules = []
        sample_smiles = []
        
        for i in range(min(100, len(train_dataset))):
            try:
                mol_feature = train_dataset.X[i]
                if hasattr(mol_feature, 'mol') and mol_feature.mol is not None:
                    mol = mol_feature.mol
                    smiles = Chem.MolToSmiles(mol)
                    sample_molecules.append(mol)
                    sample_smiles.append(smiles)
                    
                    if len(sample_molecules) >= 20:  # Limit for demo
                        break
            except:
                continue
        
        # Calculate molecular descriptors
        molecular_data = []
        for mol, smiles in zip(sample_molecules, sample_smiles):
            try:
                descriptors = {
                    'SMILES': smiles,
                    'MW': Descriptors.MolWt(mol),
                    'LogP': Crippen.MolLogP(mol),
                    'HBD': Descriptors.NumHDonors(mol),
                    'HBA': Descriptors.NumHAcceptors(mol),
                    'TPSA': Descriptors.TPSA(mol),
                    'RotBonds': Descriptors.NumRotatableBonds(mol)
                }
                molecular_data.append(descriptors)
            except:
                continue
        
        mol_df = pd.DataFrame(molecular_data)
        
        if len(mol_df) > 0:
            print(f"✅ Calculated descriptors for {len(mol_df)} molecules")
            
            # Lipinski's Rule of Five analysis
            mol_df['Lipinski_Violations'] = (
                (mol_df['MW'] > 500).astype(int) +
                (mol_df['LogP'] > 5).astype(int) +
                (mol_df['HBD'] > 5).astype(int) +
                (mol_df['HBA'] > 10).astype(int)
            )
            
            drug_like = mol_df[mol_df['Lipinski_Violations'] <= 1]
            print(f"🎯 Drug-like molecules (≤1 Lipinski violations): {len(drug_like)}/{len(mol_df)}")
            
            # Create visualization
            fig, axes = plt.subplots(2, 2, figsize=(12, 10))
            
            # MW vs LogP
            axes[0,0].scatter(mol_df['MW'], mol_df['LogP'], alpha=0.7, c='skyblue')
            axes[0,0].axhline(5, color='red', linestyle='--', alpha=0.5, label='LogP limit')
            axes[0,0].axvline(500, color='red', linestyle='--', alpha=0.5, label='MW limit')
            axes[0,0].set_xlabel('Molecular Weight')
            axes[0,0].set_ylabel('LogP')
            axes[0,0].set_title('Molecular Weight vs Lipophilicity')
            axes[0,0].legend()
            axes[0,0].grid(True, alpha=0.3)
            
            # HBD vs HBA
            axes[0,1].scatter(mol_df['HBD'], mol_df['HBA'], alpha=0.7, c='lightgreen')
            axes[0,1].axhline(10, color='red', linestyle='--', alpha=0.5, label='HBA limit')
            axes[0,1].axvline(5, color='red', linestyle='--', alpha=0.5, label='HBD limit')
            axes[0,1].set_xlabel('Hydrogen Bond Donors')
            axes[0,1].set_ylabel('Hydrogen Bond Acceptors')
            axes[0,1].set_title('Hydrogen Bonding Capacity')
            axes[0,1].legend()
            axes[0,1].grid(True, alpha=0.3)
            
            # TPSA distribution
            axes[1,0].hist(mol_df['TPSA'], bins=20, alpha=0.7, color='orange', edgecolor='black')
            axes[1,0].axvline(140, color='red', linestyle='--', linewidth=2, label='TPSA limit (140)')
            axes[1,0].set_xlabel('Topological Polar Surface Area')
            axes[1,0].set_ylabel('Number of Molecules')
            axes[1,0].set_title('TPSA Distribution')
            axes[1,0].legend()
            axes[1,0].grid(True, alpha=0.3)
            
            # Lipinski violations
            violation_counts = mol_df['Lipinski_Violations'].value_counts().sort_index()
            axes[1,1].bar(violation_counts.index, violation_counts.values, 
                         alpha=0.7, color='coral', edgecolor='black')
            axes[1,1].set_xlabel('Number of Lipinski Violations')
            axes[1,1].set_ylabel('Number of Molecules')
            axes[1,1].set_title('Lipinski Rule of Five Compliance')
            axes[1,1].grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.savefig('../results/figures/molecular_analysis.png', dpi=300, bbox_inches='tight')
            plt.show()
            
            # Save molecular data
            mol_df.to_csv('../data/molecules/molecular_descriptors.csv', index=False)
            print(f"💾 Molecular data saved to ../data/molecules/molecular_descriptors.csv")
            
            return mol_df
        else:
            print("❌ No valid molecular data extracted")
            return None
            
    except ImportError as e:
        print(f"❌ Error importing molecular libraries: {e}")
        print("Please install: pip install deepchem rdkit-pypi")
        return None
    except Exception as e:
        print(f"❌ Error in molecular analysis: {e}")
        return None

# Run molecular analysis
print("🧬➡️💊 Connecting protein variants to molecular therapeutics...")
molecular_data = setup_molecular_analysis()


In [ ]:
## 5. Research Summary & Next Steps

Let's generate a comprehensive research report and identify the most promising directions for your HCM variant research.


In [ ]:
def generate_research_summary():
    """Generate comprehensive research summary and recommendations"""
    
    print("🎯 COMPREHENSIVE RESEARCH SUMMARY")
    print("="*60)
    
    if df is not None:
        # Basic dataset statistics
        print(f"\\n📊 DATASET OVERVIEW")
        print(f"   • Total MYH7 variants analyzed: {len(df):,}")
        if 'AlphaMissense' in df.columns:
            print(f"   • AlphaMissense scores available: {df['AlphaMissense'].notna().sum():,}")
        if 'Rosetta_ddG' in df.columns:
            print(f"   • Rosetta ΔΔG values: {df['Rosetta_ddG'].notna().sum():,}")
        if 'pLDDT' in df.columns:
            print(f"   • pLDDT confidence scores: {df['pLDDT'].notna().sum():,}")
        
        # Identify high-priority variants
        if 'AlphaMissense' in df.columns:
            high_pathogenic = df[df['AlphaMissense'] > 0.8]
            print(f"\\n🚨 HIGH-PRIORITY VARIANTS (AlphaMissense > 0.8)")
            print(f"   • Found {len(high_pathogenic)} highly pathogenic variants")
            
            if len(high_pathogenic) > 0:
                print("   • Top 5 most pathogenic:")
                for i, (_, row) in enumerate(high_pathogenic.nlargest(5, 'AlphaMissense').iterrows(), 1):
                    variant = row.get('ProteinChange', 'N/A')
                    score = row.get('AlphaMissense', 0)
                    ddg = row.get('Rosetta_ddG', 'N/A')
                    if pd.notna(ddg):
                        ddg_str = f"ΔΔG={ddg:.2f}"
                    else:
                        ddg_str = "ΔΔG=pending"
                    print(f"     {i}. {variant}: {score:.3f} ({ddg_str})")
        
        # Structural analysis
        if 'pLDDT' in df.columns:
            high_conf = df[df['pLDDT'] > 90]
            low_conf = df[df['pLDDT'] < 50]
            print(f"\\n🏗️ STRUCTURAL CONFIDENCE ANALYSIS")
            print(f"   • High confidence regions (pLDDT>90): {len(high_conf)} variants")
            print(f"   • Low confidence regions (pLDDT<50): {len(low_conf)} variants")
            
            if 'AlphaMissense' in df.columns:
                high_conf_high_path = df[(df['pLDDT'] > 90) & (df['AlphaMissense'] > 0.8)]
                print(f"   • High-confidence + high-pathogenicity: {len(high_conf_high_path)} variants")
        
        # Stability analysis
        if 'Rosetta_ddG' in df.columns:
            mask = df['Rosetta_ddG'].notna()
            if mask.any():
                destabilizing = df[df['Rosetta_ddG'] > 1.0]
                stabilizing = df[df['Rosetta_ddG'] < -1.0]
                print(f"\\n⚖️ PROTEIN STABILITY ANALYSIS")
                print(f"   • Destabilizing variants (ΔΔG>1): {len(destabilizing)}")
                print(f"   • Stabilizing variants (ΔΔG<-1): {len(stabilizing)}")
                
                if 'AlphaMissense' in df.columns:
                    high_impact = df[(df['AlphaMissense'] > 0.8) & (df['Rosetta_ddG'] > 1.0)]
                    print(f"   • High-impact variants (pathogenic + destabilizing): {len(high_impact)}")
    
    # Research recommendations
    print(f"\\n🔬 RESEARCH RECOMMENDATIONS")
    print("   1. STRUCTURAL BIOLOGY:")
    print("      • Focus experimental validation on high-confidence, high-pathogenicity variants")
    print("      • Investigate domain-specific effects of mutations")
    print("      • Use molecular dynamics simulations for detailed mechanism studies")
    
    print("\\n   2. FUNCTIONAL STUDIES:")
    print("      • Prioritize variants with extreme ΔΔG values for in vitro analysis")
    print("      • Study effects on myosin motor function and ATP hydrolysis")
    print("      • Investigate calcium sensitivity changes")
    
    print("\\n   3. CLINICAL CORRELATION:")
    print("      • Map pathogenicity scores to patient outcomes")
    print("      • Develop variant-specific risk stratification")
    print("      • Consider genetic modifier effects")
    
    print("\\n   4. THERAPEUTIC DEVELOPMENT:")
    print("      • Screen small molecules targeting destabilized variants")
    print("      • Explore protein chaperone therapies")
    print("      • Investigate myosin modulators (like mavacamten)")
    
    print("\\n   5. COMPUTATIONAL EXTENSIONS:")
    print("      • Implement AlphaFold3 for improved accuracy")
    print("      • Add protein-protein interaction analysis")
    print("      • Integrate evolutionary conservation data")
    
    # Next steps
    print(f"\\n🚀 IMMEDIATE NEXT STEPS")
    print("   1. Run the fixed Rosetta analysis: `cd .. && bash run_rosetta_fixed.sh`")
    print("   2. Validate top variants with molecular dynamics simulations")
    print("   3. Set up automated pipeline for new variant analysis")
    print("   4. Integrate with clinical databases for outcome correlation")
    print("   5. Develop machine learning models for pathogenicity prediction")
    
    # Output files summary
    print(f"\\n📁 GENERATED FILES")
    print("   • Comprehensive analysis notebook: notebooks/1_MYH7_variant_analysis.ipynb")
    print("   • Fixed Rosetta script: run_rosetta_fixed.sh")
    print("   • Mutation file: top_variants_fixed.txt")
    print("   • Visualizations: results/figures/comprehensive_myh7_analysis.png")
    if molecular_data is not None:
        print("   • Molecular analysis: results/figures/molecular_analysis.png")
    
    print(f"\\n🎉 ANALYSIS COMPLETE!")
    print("Your MYH7 variant pipeline is now ready for production research!")

# Generate the final summary
generate_research_summary()
